In [1]:
import numpy as np
import matplotlib.pyplot as plt              #For analysis and plotting
import sys

import PySpice
import PySpice.Logging.Logging as Logging
from PySpice.Spice.Netlist import Circuit    #For creating circuits
from PySpice.Unit import *                   #For adding units

logger = Logging.setup_logging()

In [2]:
#Functions
def format_output(analysis):
    sim_res_dict = {} #create dictionary
    for node in analysis.nodes.values():
        data_label = "%s" % str(node) #extract node name
        sim_res_dict[data_label] = np.array(node)
    return(sim_res_dict)

In [3]:
def funcion_pyspice(Matriz1, V_total, rho_res, Nc, Mf):
    N_res =((Nc*(Mf-1))+(Mf*(Nc-1)))
    Nnodos = (Nc*Mf)
    
    circuit = Circuit('Red_mem')
    circuit.V('Fuente', 1, circuit.gnd, V_total@u_V)       #Fuente, llamada 'Fuente', conectada al nodo_1 y a gnd(0), con un valor de 10V

    #En la definicion del Netlist se puede cambiar la posicion de GND
    for i in range(0, N_res):
        if (i==(N_res-Nc)):
            circuit.R(Matriz1[i][0], Matriz1[i][1], circuit.gnd,   rho_res[i]@u_kΩ)
        elif (i==(N_res-1)):
            circuit.R(Matriz1[i][0], Matriz1[i][1], circuit.gnd,   rho_res[i]@u_kΩ)
        else:
            circuit.R(Matriz1[i][0], Matriz1[i][1], Matriz1[i][2], rho_res[i]@u_kΩ)
    
    #print('Circuit Netlist', circuit)
    #print(Matriz1)
    
    simulator = circuit.simulator()   
    analysis = simulator.operating_point()
    resultados = format_output(analysis)           #Le damos formato a los resultados
    resultados[str(Nnodos)] = np.array([0.])       #Le agrego el nodo GND (el ultimo) que por algun motivo no aparece dentro de los resultados
    #print(resultados)
    #print(' ')

    #for i in range(1, Nnodos+1):
        #print('Nodo '+str(i), resultados[str(i)])   #Tension en cada nodo
    #print(' ')

    deltav=list(range(0, N_res))
    for i in range(0, N_res):
        deltav[(i)]= (float(resultados[str(Matriz1[i][1])]))-(float(resultados[str(Matriz1[i][2])]))  

    return(deltav)